# CATNIP Analysis of light sheet imaging of whole-brain c-Fos immunostaining in constitutive PACAP-knockout and control PACAP-flox/flox mice

The following Dandiset contains deconvolved and homogeneity corrected images of c-FOS immunostained whole mouse brains and the derivatives from the [CATNIP](https://github.com/snehashis-roy/CATNIP) analysis pipeline.

If running on [DandiHub](https://hub.dandiarchive.org/), please select the `dandi-dandi-openscope` kernel. Please note the `k3d` visualization will not work on DandiHub.

In [ ]:
import pandas as pd
from dandi.dandiapi import DandiAPIClient
import fsspec
import tifffile
from pathlib import Path
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np


## Create a class for downloading files from CATNIP Dandiset

DANDI Archive stores assests in S3 buckets and provides an API to access those clients. In order to facilitate downloading different assests from the CATNIP dataset we will construct a CatNabber class. CatNabber provides a way to easily download specific files from a DANDI dataset to a local directory, handling the retrieval of download URLs and checking for existing files.

In [ ]:
class CatNabber:
    def __init__(self, dandiset_id: str, local_root: Path = Path("./catnip")):
        self.dandiset_id: str = dandiset_id
        if not local_root.exists():
            local_root.mkdir(exist_ok=True, parents=True)
        self.local_root: Path = local_root


    def _get_dandi_s3_uri(self, dandi_filepath: str) -> str:
        with DandiAPIClient() as client:
            asset = client.get_dandiset(dandiset_id, 'draft').get_asset_by_path(dandi_filepath)
            s3_url = asset.get_content_url(follow_redirects=1, strip_query=True)
        return s3_url


    def download_file(self, dandi_filepath: Path | str, force_redownload: bool = False) -> Path:
        s3_uri: str = self._get_dandi_s3_uri(dandi_filepath=str(dandi_filepath))
        local_filepath: Path = self.local_root.joinpath(dandi_filepath)
        local_filepath.parent.mkdir(exist_ok=True, parents=True)
        if local_filepath.exists() and force_redownload == False:
            print(f"{local_filepath} already exists. Skipping download")
            return local_filepath
        try:
            with fsspec.open(s3_uri, 'rb') as f_s3:
                with open(local_filepath, 'wb') as f_local:
                    f_local.write(f_s3.read())
            print(f"File '{s3_uri}' downloaded")
        except Exception as e:
            print(f"Error downloading file: {e}")
        return local_filepath


## CATNIP Dandiset Samples

The CATNIP dataset is in [BIDS format](https://bids.neuroimaging.io/index.html), specifically the [BIDS microscopy extension](https://bids-specification.readthedocs.io/en/stable/modality-specific-files/microscopy.html). Part of the CATNIP dataset is a list samples and their corresponding participant ids. Below, we will instantiate a CatNabber class with the CATNIP dandiset id `001362` and download the `samples.tsv` file. 

In [ ]:
dandiset_id="001362"
local_root: Path = Path("./catnip")
catnabber = CatNabber(dandiset_id, local_root)
sample_path: str = "samples.tsv"
local_sample: Path = catnabber.download_file(sample_path)


Below, we will show the samples in the CATNIP dandiset using a Pandas DataFrame. Each sample is an image stack of either the left or right hemisphere a participant's brain. Note, not all participants have both hemispheres imaged. The genotype of the participant is listed under the `pathology` column.

In [ ]:
samples: pd.DataFrame = pd.read_csv(local_sample, sep='\t')


In [ ]:
samples


In [ ]:
print(f"There are {len(samples)} in this dataset with {len(samples['participant_id'].unique())} participants")


## Deconvolved and Inhomogeneity Corrected Images

The [BIDS microscopy extension](https://bids-specification.readthedocs.io/en/stable/modality-specific-files/microscopy.html) requires a strict directory structure. Using this structure we can gather a list of all the deconvolved and inhomogeneity corrected image stacks for each sample listed in the `samples.tsv` file.

In [ ]:
corrected_images_paths: list[Path] = []
for i, row in samples.iterrows():
    corrected_image_parent: Path = Path(f"{row['participant_id']}").joinpath("micr")
    corrected_image_filename: str = row["participant_id"] + "_" +  row["sample_id"] + "_SPIM.ome.btf"
    corrected_image_filepath: Path = corrected_image_parent.joinpath(corrected_image_filename)
    corrected_images_paths.append(corrected_image_filepath)
    print(f"{i}: {corrected_image_filepath}")


Each of this image stacks are approximately 7 GB. These images can be downloaded from the S3 bucket to the DandiHub server hosting this notebook in a few minutes. Downloading them to a local machine can take considerably longer depending on your internet connection.

In [ ]:
local_image = catnabber.download_file(corrected_images_paths[0])


Using DandiHub's large server, the entire image stack can easily be loaded into memory using the `tifffile` module.

In [ ]:
im_array = tifffile.imread(local_image)


For sample `sub-45424flox_sample-LeftHemiSphere` the resulting `im_array` has the dimensions ZYX:

In [ ]:
im_shape = im_array.shape
im_shape


In order to display only planes with signal, we will trim off the blank planes.

In [ ]:
z_offset = -1
plane = im_array[z_offset, :,:]
while plane.sum() == 0:
    z_offset -= 1
    plane = im_array[z_offset, :, :]
print(z_offset)


## Visualizing the image stack

Below the code takes 9 evenly spaced Z-planes from the stack excluding the blank planes.

In [ ]:
# Determine the z-steps for the 9 images
z_size, x_size, y_size = im_array.shape
num_images = 9
z_indices = np.linspace(0, z_size + z_offset, num_images, dtype=int)

# Create a 3x3 subplot grid
fig, axes = plt.subplots(3, 3, figsize=(10, 10))
fig.suptitle(f"{samples['participant_id'][0]}: {samples['sample_id'][0]}\nDeconvolved and N4 Corrected")

# Flatten the axes array for easy indexing
axes = axes.flatten()

# Iterate through the z-indices and display the corresponding slices
for i, z_index in enumerate(z_indices):
    # Extract the 2D image slice
    img_slice = im_array[z_index, :, :]

    # Display the image in the corresponding subplot
    axes[i].imshow(img_slice, cmap='gray')  # You can change 'gray' to other colormaps
    axes[i].set_title(f'Z = {z_index}')
    axes[i].axis('off')  # Turn off axis labels and ticks

# Adjust layout to prevent overlapping titles
plt.tight_layout()

# Show the plot
plt.show()

# Delete im_array to free up RAM
del im_array


## Allen Mouse Brain Atlas Registration

The deconvolved and inhomogeneity corrected images are registered to the [Allen Mouse Brain Atlas](https://mouse.brain-map.org/static/atlas). The resulting segmentation maps have the same dimensions as the corrected image stack. The maps are found in the `AtlasLabel` derivatives folder and are only approximately 50 MB. First we will download the segmentation map corresponding to sample `sub-45424flox_sample-LeftHemiSphere` shown above.

In [ ]:
dandi_atlas_path: str = r"derivatives/AtlasLabel/sub-45424flox/micr/sub-45424flox_sample-LeftHemisphere_space-orig_dseg.ome.btf"
local_atlas = catnabber.download_file(dandi_atlas_path)
atlas_array = tifffile.imread(local_atlas)
atlas_shape = atlas_array.shape
print(f"Image stack shape: {im_shape}, Atlas stack shape: {atlas_shape}")


The CATNIP dandiset has a `dseg.tsv` file that indicates the brain region for each index value and a corresponding color hexvalue.

In [ ]:
dandi_atlas_colormap_path: str = r"derivatives/AtlasLabel/dseg.tsv"
local_atlas_colormap = catnabber.download_file(dandi_atlas_colormap_path)
atlas_colormap_df: pd.DataFrame = pd.read_csv(local_atlas_colormap, sep='\t')
atlas_colormap_df


Below, we convert the `color` values in the `dseg.tsv` file to a colormap. We must append black, `#000000`, to the beginning of the list for unmapped pixels.

In [ ]:
color_list: list[str] = ["#000000"] + atlas_colormap_df['color'].to_list()
atlas_colormap: ListedColormap = ListedColormap(color_list)


## Visualizing the Atlas Segmentation Map

Below we show the same planes fo the Atlas Segmentation Map as the corrected image above.

In [ ]:
# Determine the z-steps for the 9 images
z_size, x_size, y_size = atlas_array.shape
num_images = 9
z_indices = np.linspace(0, z_size + z_offset, num_images, dtype=int)

# Create a 3x3 subplot grid
fig, axes = plt.subplots(3, 3, figsize=(10, 10))
fig.suptitle(f"{samples['participant_id'][0]}: {samples['sample_id'][0]}\nAllen Mouse Brain Atlas")

# Flatten the axes array for easy indexing
axes = axes.flatten()

# Iterate through the z-indices and display the corresponding slices
for i, z_index in enumerate(z_indices):
    # Extract the 2D image slice
    img_slice = atlas_array[z_index, :, :]

    # Display the image in the corresponding subplot
    axes[i].imshow(img_slice, cmap=atlas_colormap)  # You can change 'gray' to other colormaps
    axes[i].set_title(f'Z = {z_index}')
    axes[i].axis('off')  # Turn off axis labels and ticks

# Adjust layout to prevent overlapping titles
plt.tight_layout()

# Show the plot
plt.show()

# Delete the atlas_array to free up RAM
del atlas_array


## c-FOS Signal Segmentation Masks

c-FOS signals are segmented using range of threshold values. These result in a series of binary images. Below, we will iterate over the binary masks for plane 449 shown in the center of the plot above. The threshold values range from 4000 to 32000 with a step size of 2000 units. First, we must download all the binary image stacks for sample `sub-45424flox_sample-LeftHemisphere`. Each image stack file is between 7 and 21 MB.


In [ ]:
binary_map: dict[int, Path] = {}
for thresh_int in range(4000, 34000, 2000):
    thresh_str: str = str(thresh_int).zfill(6)
    dandi_atlas_path: str = f"derivatives/FastRadialSymmetryTransformSegmentation/sub-45424flox/micr/sub-45424flox_sample-LeftHemisphere_acq-{thresh_str}_SPIM.ome.btf"
    local_path: Path = catnabber.download_file(dandi_atlas_path)
    binary_map[thresh_int] = local_path


## Visualizing the Segmentastion Masks

In order to compare the masks of the same plan across different thresholds, we will iterate over the images and only show plan `449` in the subplots.

In [ ]:
# Create a 3x5 subplot grid
fig, axes = plt.subplots(3, 5, figsize=(15, 10))
fig.suptitle(f"{samples['participant_id'][0]}: {samples['sample_id'][0]}\nBinary c-FOS signals")

# Flatten the axes array for easy indexing
axes = axes.flatten()

# Iterate through the z-indices and display the corresponding slices
for i, thresh_int in enumerate(range(4000, 34000, 2000)):

    # Extract the 2D image slice
    binary_array = tifffile.imread(binary_map[thresh_int])
    binary_image = binary_array[449,:,:]
    del binary_array
    

    # Display the image in the corresponding subplot
    axes[i].imshow(binary_image, cmap='gray', vmin=0, vmax=1)
    axes[i].set_title(f'Threshold={thresh_int}')
    axes[i].axis('off')  # Turn off axis labels and ticks


## Quantification of Signal by Region

The number of c-FOS signals were counted per region for each threshold value. These are present in the Dandiset as TSV files in the `FastRadialSymmetryTransformSegmentationCounts` derivative folder. Below are the counts for sample `sub-45424flox_sample-LeftHemisphere` at a threshold vlaue of `4000`.

In [ ]:
counts_path: str = f"derivatives/FastRadialSymmetryTransformSegmentationCounts/sub-45424flox/micr/sub-45424flox_sample-LeftHemisphere_acq-004000_SPIM.tsv"
local_counts: Path = catnabber.download_file(counts_path)
counts: pd.DataFrame = pd.read_csv(local_counts, sep='\t', index_col=False)


In [ ]:
counts


## Heatmaps of the Cell Counts

Heatmaps of the cell counts were generated the cell counts. These heatmaps are downsampled and can be viewed as volumes using the `k3d` module on DandiHub or on a Jupyter Notebook. First, we download the image to the server using CatNabber.

In [ ]:
heatmap_image = catnabber.download_file("derivatives/HeatmapsAtlasSpace/sub-45424flox/micr/sub-45424flox_sample-LeftHemisphere_acq-heatmap_res-25um_SPIM.ome.btf")


Now we load the image into memory. Note the much smaller size of the heatmap array.

In [ ]:
heatmap_array = tifffile.imread(heatmap_image)
heatmap_array.shape


## Volume Visualization using K3D in Jupyter Notebook

### K3D no longer working on DANDI HUB

On DandiHub, the Jupyter server no longer has the [k3d Jupyter Extension](https://github.com/K3D-tools/K3D-jupyter) installed. This prevents 3D visualization of the heatmap array. Please view `README.md` to install a local Jupyter Notebook server for k3d visualizations.


The volume can be rotated, zoomed, and further manipulated using the `K3D panel` in the top right corner. Please see the [panel user guide](https://k3d-jupyter.org/user/panel.html). Finally, we use [Google's Turbo](https://research.google/blog/turbo-an-improved-rainbow-colormap-for-visualization/) colormap to provide contrast.

In [ ]:
import k3d
from k3d.colormaps import matplotlib_color_maps

k3d.factory.volume(volume=heatmap_array, color_map=matplotlib_color_maps.Turbo)
